In [35]:
import pandas as pd
import numpy as np
import torch
from torch import nn

In [7]:
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [122]:
df = pd.read_csv("PERFORMANCE_FIX.csv")

In [123]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [124]:
def remove_non_ascii(s):
    return "".join(c for c in s if ord(c)<128)

In [125]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [126]:
df['Text'] = df['Text'].astype(str)
df['Text'] = df['Text'].str.lower()
df['Text'] = df['Text'].apply(lambda x: remove_punct(x))
df['Text'] = df['Text'].apply(lambda x: remove_non_ascii(x))
df['Text'] = df['Text'].str.replace('â€™', '\'')
df['Text'] = df['Text'].apply(remove_emojis)

In [127]:
def tokenization(text):
  text = word_tokenize(text)
  return text

In [128]:
stop_words = nltk.corpus.stopwords.words('english') 
def stopwords(text):
  text = [word for word in text if word not in stop_words]
  return text

In [129]:
df['Tokenized'] = df['Text'].apply(tokenization)
df['Stopwords'] = df['Tokenized'].apply(stopwords)

In [130]:
def joined(text):
  text = " ".join(text)
  return text

In [131]:
df['Stopwords'] = df['Stopwords'].apply(lambda x: joined(x))

In [79]:
!pip install spacy
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-10-14 17:09:49.897526: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 7.9 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [132]:
import spacy

In [133]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [134]:
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [135]:
df['Vectorizer'] = df['Stopwords'].apply(lambda x: get_vec(x))

In [136]:
df['Sentiment'].value_counts()

2    740
0    729
1    715
Name: Sentiment, dtype: int64

In [137]:
final = pd.DataFrame({'Vector':df['Vectorizer'], 'Sentiment':df['Sentiment']})
final.head()

,Vector,Sentiment
0,"[-0.84328747, 1.194625, -2.562034, 0.5952651, ...",0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
2,"[-0.49973536, 0.56508344, -3.2303066, -0.62693...",1
3,"[-1.1556596, 0.2174886, -1.0837786, -0.7909443...",1
4,"[-0.2507867, -0.31328002, -1.7633333, -1.35640...",0


In [138]:
X = final['Vector'].to_numpy()
X = X.reshape(-1, 1)

In [139]:
import numpy as np
X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)
y = final['Sentiment'].to_numpy()

In [140]:
len(X), len(y)

(2184, 2184)

In [141]:
X[0].shape, y[0].shape

((300,), ())

In [142]:
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.LongTensor)

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [150]:
class classification(nn.Module):
  def __init__(self):
    super().__init__()
    self.lstm = nn.LSTM(300, 10)
    self.layer_1 = nn.Sequential(
        nn.ReLU(),
        nn.Linear(in_features = 10, out_features = 10),
        nn.ReLU(),
        nn.Linear(in_features = 10, out_features = 3)
    )
  
  def forward(self, x):
    out, _ = self.lstm(x)
    out = self.layer_1(out)
    return out

model = classification()

In [151]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters(),
                            lr = 0.1)

In [152]:
torch.unique(y)

tensor([0, 1, 2])

In [147]:
# Calculate accuracy - out of 100 examples, what percentage does our model get right?
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc

In [155]:
# Fit the multi-class model to the data
torch.manual_seed(42)

# Set number of epochs
epochs = 5000

# Loop through the data
for epoch in range(epochs):
  model.train()
  y_logits = model(X_train)
  y_pred = torch.softmax(y_logits, dim = 1).argmax(dim=1)
  loss = loss_fn(y_logits, y_train.type(torch.LongTensor))
  acc = accuracy_fn(y_true=y_train,
                    y_pred=y_pred)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  ### Testing
  model.eval()
  with torch.inference_mode():
    test_logits = model(X_test)
    test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
    test_loss = loss_fn(test_logits, y_test.type(torch.LongTensor))
    test_acc = accuracy_fn(y_true=y_test,
                      y_pred=test_pred)
  # Print out what's happening
  if epoch % 100 == 0:
    print(f"Epoch : {epoch} | Loss : {loss:.5f}, Accuracy : {acc:.2f}% | Test Loss : {test_loss:.5f}, Test Accuracy : {test_acc:.2f}%")

Epoch : 0 | Loss : 0.56343, Accuracy : 77.05% | Test Loss : 0.90054, Test Accuracy : 62.24%
Epoch : 100 | Loss : 0.49401, Accuracy : 81.05% | Test Loss : 0.91336, Test Accuracy : 63.16%
Epoch : 200 | Loss : 0.43624, Accuracy : 84.09% | Test Loss : 0.94341, Test Accuracy : 64.07%
Epoch : 300 | Loss : 0.39586, Accuracy : 85.98% | Test Loss : 0.97853, Test Accuracy : 64.07%
Epoch : 400 | Loss : 0.34885, Accuracy : 87.86% | Test Loss : 1.02363, Test Accuracy : 62.47%
Epoch : 500 | Loss : 0.30757, Accuracy : 89.47% | Test Loss : 1.08543, Test Accuracy : 62.47%
Epoch : 600 | Loss : 0.26632, Accuracy : 91.76% | Test Loss : 1.07657, Test Accuracy : 63.16%
Epoch : 700 | Loss : 0.23254, Accuracy : 92.73% | Test Loss : 1.12003, Test Accuracy : 62.47%
Epoch : 800 | Loss : 0.21018, Accuracy : 93.76% | Test Loss : 1.15051, Test Accuracy : 63.39%
Epoch : 900 | Loss : 0.18739, Accuracy : 94.73% | Test Loss : 1.21456, Test Accuracy : 62.47%
Epoch : 1000 | Loss : 0.29684, Accuracy : 87.64% | Test Loss :

In [156]:
model.eval()
with torch.inference_mode():
  y_logits = model(X_test)

In [157]:
y_pred_probs = torch.softmax(y_logits, dim=1)
y_preds = torch.argmax(y_pred_probs, dim=1)
y_preds[:10]

tensor([0, 2, 1, 0, 1, 0, 1, 0, 0, 2])